<a href="https://colab.research.google.com/github/Saym0310/Sanjeev/blob/main/Ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 2 Notebook

### Connect to Finnhub WebSocket for Real-time Stock Trades

To stream data from Finnhub, you need an API key. Please follow these steps:

1.  **Get an API Key**: Sign up at [finnhub.io](https://finnhub.io/) to obtain your free API key.
2.  **Store in Colab Secrets**: In Google Colab, click on the '🔑' icon (Secrets) in the left sidebar.
3.  **Add Secret**: Add a new secret with the name `FINNHUB_API_KEY` and paste your API key as the value.

Once your API key is configured, run the following code cell to connect to Finnhub and stream real-time trade data for AAPL (Apple Inc.).

# Imports the Library

In [1]:
import asyncio
import websockets
import pandas as pd
import json
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from datetime import datetime
from google.colab import userdata

# Finnhub streaming

In [2]:
streaming_df = pd.DataFrame(columns=['Symbol', 'Price', 'Volume', 'Timestamp'])

async def connect_and_stream_finnhub_dataframe():
    # Retrieve Finnhub API key from Colab secrets
    finnhub_api_key = userdata.get('FINNHUB_API_KEY')
    if not finnhub_api_key:
        print("Error: FINNHUB_API_KEY not found in Colab secrets. Please add it.")
        return

    # Finnhub WebSocket API for real-time stock trades
    # We'll subscribe to AAPL (Apple Inc.) for demonstration
    uri = f"wss://ws.finnhub.io?token={finnhub_api_key}"
    print(f"Connecting to Finnhub WebSocket for AAPL trades...")

    # Initialize an empty DataFrame to store streaming data
    streaming_df = pd.DataFrame(columns=['Symbol', 'Price', 'Volume', 'Timestamp'])
    print("Initialized empty DataFrame for Finnhub trades.")
    display(streaming_df) # Display initial empty DataFrame

    trade_count = 0
    max_trades = 10 # Limit to 10 trades for demonstration

    try:
        async with websockets.connect(uri) as websocket:
            print("Connection established. Subscribing to AAPL...")
            # Subscribe to AAPL trades
            subscribe_message = {"type":"subscribe", "symbol":"AAPL"}
            await websocket.send(json.dumps(subscribe_message))
            print("Subscription message sent.")

            while trade_count < max_trades:
                try:
                    message = await asyncio.wait_for(websocket.recv(), timeout=10) # Set a timeout
                    data = json.loads(message)

                    # Finnhub sends different types of messages, we're interested in 'trade' data
                    if data.get('type') == 'trade' and 'data' in data:
                        for trade in data['data']:
                            symbol = trade.get('s')
                            price = trade.get('p')
                            volume = trade.get('v')
                            # Finnhub timestamp is Unix timestamp in milliseconds, convert to seconds
                            timestamp = datetime.fromtimestamp(trade.get('t') / 1000).strftime('%Y-%m-%d %H:%M:%S')

                            # Create new data row
                            new_data_row = pd.DataFrame({
                                'Symbol': [symbol],
                                'Price': [price],
                                'Volume': [volume],
                                'Timestamp': [timestamp]
                            })
                            streaming_df = pd.concat([streaming_df, new_data_row], ignore_index=True)
                            trade_count += 1

                            # Clear previous output and display the updated DataFrame
                            clear_output(wait=True)
                            display(streaming_df)

                            print(f"Received trade {trade_count}/{max_trades}: {symbol} Price: {price}, Volume: {volume}")

                            if trade_count >= max_trades:
                                break # Exit inner loop if max_trades reached

                    elif data.get('type') == 'ping':
                        # Finnhub sends ping messages to keep connection alive
                        print("Received ping from Finnhub.")
                    else:
                        # Other message types (e.g., initial ack of subscription)
                        print(f"Received non-trade message: {data.get('type')}")

                except asyncio.TimeoutError:
                    print("No message received within timeout. Reconnecting...")
                    break # Break to allow reconnection attempt
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e} - Message: {message[:100]}")
                except Exception as e:
                    print(f"An unexpected error occurred: {e}")
                    break

    except websockets.exceptions.ConnectionClosedOK:
        print("WebSocket connection closed normally.")
    except Exception as e:
        print(f"Finnhub WebSocket connection error: {e}")

    print(f"Finished streaming {trade_count} trades.")

# To run this in a Colab notebook, you can use:
streaming_df = await connect_and_stream_finnhub_dataframe()
streaming_df



,Symbol,Price,Volume,Timestamp
0,AAPL,257.93,100,2026-01-29 18:39:22
1,AAPL,257.93,100,2026-01-29 18:39:22
2,AAPL,257.93,100,2026-01-29 18:39:23
3,AAPL,257.93,100,2026-01-29 18:39:23
4,AAPL,257.93,100,2026-01-29 18:39:23
5,AAPL,257.93,100,2026-01-29 18:39:23
6,AAPL,257.93,181,2026-01-29 18:39:23
7,AAPL,257.93,100,2026-01-29 18:39:23
8,AAPL,257.93,100,2026-01-29 18:39:23
9,AAPL,257.93,100,2026-01-29 18:39:23


Received trade 10/10: AAPL Price: 257.93, Volume: 100
Finished streaming 10 trades.


# Plot the streamed AAPL prices

In [3]:
import matplotlib.pyplot as plt

if streaming_df is None:
    print("streaming_df is None – no data to plot.")
elif streaming_df.empty:
    print("streaming_df is empty – no data to plot.")
else:
    plt.figure(figsize=(8, 4))

    plt.plot(
        streaming_df["Timestamp"],
        streaming_df["Price"],
        marker="o"
    )

    plt.title("AAPL Stock Price (Finnhub Streaming)")
    plt.xlabel("Timestamp")
    plt.ylabel("Price (USD)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


streaming_df is None – no data to plot.
